<a href="https://colab.research.google.com/github/dhyaks/Assigment/blob/main/M20AIE210.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder=LabelEncoder()
import zipfile



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import zipfile

In [ ]:
zip_ref = zipfile.ZipFile('/content/drive/MyDrive/BIO_VISION/predict-20221101T092253Z-001.zip', 'r')
zip_ref.extractall('/content/drive/MyDrive/BIO_VISION')
zip_ref.close()
zip_ref = zipfile.ZipFile('/content/drive/MyDrive/BIO_VISION/train-20221101T092401Z-001.zip', 'r')
zip_ref.extractall('/content/drive/MyDrive/BIO_VISION')
zip_ref.close()
zip_ref = zipfile.ZipFile('/content/drive/MyDrive/BIO_VISION/test-20221101T092219Z-001.zip', 'r')
zip_ref.extractall('/content/drive/MyDrive/BIO_VISION')
zip_ref.close()

In [ ]:
################Image_counts_and_load_in_dataFrame######

In [ ]:
import glob
import pandas as pd
import os

In [ ]:
def data_image_data_create(f):
  image_file=f+'/'+'*'
  path,file=os.path.split(f)
  l1=[]
  count=0
  for ii in glob.glob(image_file):
    l1.append(ii)
    count=count+1
  data=pd.DataFrame({'filename':l1})
  data['Category']=str(file)
  data['Category']=data['Category'].str.lower()
  print('**Folder Name=',file,"**" 'Image Count=',count)
  del l1
  return data

def data_extract(path):
  f_data=pd.DataFrame()
  p=path+'/'+'*'
  for i in glob.glob(p):
    d=data_image_data_create(i)
    f_data=f_data.append(d)
  return f_data

In [ ]:
train_data=data_extract('/content/drive/MyDrive/BIO_VISION/train')

In [ ]:
test_data=data_extract('/content/drive/MyDrive/BIO_VISION/test')

In [ ]:
###############decode your Category###########

In [ ]:
train_data['Category'].value_counts()

In [ ]:
train_data['Category_Label']=label_encoder.fit_transform(train_data['Category'])
d1=train_data[['Category','Category_Label']].drop_duplicates()

In [ ]:
d1

In [ ]:
test_data['Category'].value_counts()

In [ ]:
test_data['Category_Label']=label_encoder.fit_transform(test_data['Category'])
d2=test_data[['Category','Category_Label']].drop_duplicates()

In [ ]:
d2

In [ ]:
###########Image Size##############

In [ ]:
import cv2

In [ ]:
def image_shape(f):
  im=cv2.imread(f)
  size=im.shape
  return size

In [ ]:
def image_size_count(data):
  size_data=[]
  for i in data['filename']:
    im=cv2.imread(i)
    try:
      x=im.shape 
      size_data.append(x)
    except:
      print(i)
  df=pd.DataFrame({'Size':size_data})
  return df

In [ ]:
train_size=image_size_count(train_data)

In [ ]:
train_size['Size'].value_counts()

In [ ]:
test_size=image_size_count(test_data)

In [ ]:
test_size['Size'].value_counts()

In [ ]:
##################Resize_Image######

In [ ]:
train_data

In [ ]:
def resize_image(data):
  for i in data['filename']:
    try:
      img=cv2.imread(i)
      print('Original Dimensions: ',img.shape)
      width=int(150)
      height=int(150)
      dim=(width,height)
      resized=cv2.resize(img,dim,interpolation=cv2.INTER_AREA)
      print('Resized Dimensions: ',resized.shape)
      cv2.imwrite(i,resized)
    except:
      print(i)

In [ ]:
resize_image(train_data)

In [ ]:
resize_image(test_data)

In [ ]:
resize_train_size=image_size_count(train_data)

In [ ]:
#################Show One Image In very Folder##########

In [ ]:
from PIL import Image

In [ ]:
lst=train_data['Category'].unique()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


In [ ]:
for i in lst:
  res=train_data[train_data['Category']==i]
  res1=res[:5]
  x=res1['Category'][0]
  for i in res1['filename']:
    img = mpimg.imread(i)
    imgplot = plt.imshow(img)
    plt.show()
    print(x)


In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout

In [ ]:
train_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/drive/MyDrive/BIO_VISION/train',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(150,150)
)

In [ ]:
validation_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/drive/MyDrive/BIO_VISION/test',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(150,150)
)

In [ ]:
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)

In [ ]:
model = Sequential()

model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(150,150,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1,activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history = model.fit(train_ds,epochs=100,validation_data=validation_ds)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'],color='red',label='train')
plt.plot(history.history['val_accuracy'],color='blue',label='validation')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'],color='red',label='train')
plt.plot(history.history['val_loss'],color='blue',label='validation')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'],color='red',label='train')
plt.plot(history.history['val_loss'],color='blue',label='validation')
plt.legend()
plt.show()

In [ ]:
##################Predict#######################

In [ ]:
f=glob.glob('/content/drive/MyDrive/BIO_VISION/predict/*')

In [ ]:
l1=[]
for i in f:
  l1.append(i)
predit_data=pd.DataFrame({'filename':l1})


In [ ]:
resize_image(predit_data)